In [1]:
!pip install transformers
!pip install ipdb
!pip install datasets
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.1

In [5]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from google.colab import drive, files
import torch
import os
import pandas as pd
device = torch.device('cpu')
model_name_or_path = "t5-base"
print(device)
import random
from datasets import Dataset
import numpy as np
from transformers import DataCollatorForSeq2Seq
from torch.nn import functional as F
import numpy as np
import nltk

cpu


In [3]:
drive.mount('/content/drive')
root_dir = "drive/My Drive/Colab Notebooks/"
data_dir = "drive/My Drive/Colab Notebooks/"
file_1 = "simple_wh_extraction.csv" 
file_2 = "adjunct_island.csv"
file_3 = "wh_island.csv"
file_4 = "complexNP_island.csv"
file_5 = "mix_construction.csv"
file_6 = "perplexity_corpus.csv"

df1 = pd.read_csv(data_dir + file_1)
df2 = pd.read_csv(data_dir + file_2)
df3 = pd.read_csv(data_dir + file_3)
df4 = pd.read_csv(data_dir + file_4)
df5 = pd.read_csv(data_dir + file_5)
train_df = [df1, df2, df3, df4, df5]
per_df = pd.read_csv(data_dir + file_6)
# train_df = pd.concat([df1, df2, df3])
# print(len(train_df))
# file_1 = "wh-island_input.xlsx"
# file_2 = "wh-island_input2.xlsx"
# eval_df = pd.read_excel(root_dir+file_1)

Mounted at /content/drive


In [ ]:
def mask_random_token(sentence, tokenizer):
    tokens = tokenizer.tokenize(sentence)
    if len(tokens) > 1:
        mask_index = random.randint(0, len(tokens) - 1)
        masked_tokens = tokens[:mask_index] + ['<extra_id_0>'] + tokens[mask_index + 1:]
        target_sentence = "<extra_id_0> " + tokens[mask_index]
        return tokenizer.convert_tokens_to_string(masked_tokens), target_sentence
    else:
        return sentence, "<extra_id_0> " + tokens[0]


def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = [item for item in sample["input"]]
    targets = [item for item in sample["output"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


class MaskedWordPredictionDataset(Dataset):
    def __init__(self, masked_sentences, target_sentences):
        self.masked_sentences = masked_sentences
        self.target_sentences = target_sentences

    def __len__(self):
        return len(self.masked_sentences)

    def __getitem__(self, idx):
        masked_sentence = self.masked_sentences[idx]
        target_sentence = self.target_sentences[idx]
        return {
            "input": masked_sentence,
            "output": target_sentences,
        }

sentence_columns = ['obj_that', 'obj_wh', 'pp_wh', 'subj_wh', 'adj_that', 'adj_wh', 
                    'adj_subj_that', 'adj_subj_wh', 'emb_that', 'emb_wh', 'emb_whether', 'np_that', 'np_wh']
all_sentences = []

for i in range(5):
    sentences = []
    df = train_df[i]
    for key in df.keys():
        if key not in sentence_columns:
            continue
        sentences.extend(df[key].tolist()[:250])
    all_sentences.append(sentences)
    print(len(sentences))
settings = ["ft_simple", "ft_adjunct", "ft_wh", "ft_complex", "ft_mix"]




# print(len(simple_sentences))
# print(len(adj_sentences))
# print(len(wh_sentences))

1000
1000
750
500
845


In [ ]:
# Fine-tuning T5
# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100

for i in range(5):
    setting = settings[i]
    ft_sentences = all_sentences[i]

    model = T5ForConditionalGeneration.from_pretrained(model_name_or_path)
    model = model.to(device)

    # Data collator
    data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=label_pad_token_id,
        pad_to_multiple_of=8
    )

    # Create the dataset
    tokenized_inputs = [tokenizer(sentence) for sentence in ft_sentences]
    input_lengths = [len(x["input_ids"]) for x in tokenized_inputs]
    max_source_length = int(np.percentile(input_lengths, 100))
    max_target_length = 5
    masked_and_target_sentences = [mask_random_token(sentence, tokenizer) for sentence in ft_sentences]
    masked_sentences, target_sentences = zip(*masked_and_target_sentences)
    masked_sentences = list(masked_sentences)
    target_sentences = list(target_sentences)
    masked_dataset = MaskedWordPredictionDataset(masked_sentences, target_sentences)
    data = {"input": masked_sentences, "output": target_sentences}
    hf_dataset = Dataset.from_dict(data)
    tokenized_dataset = hf_dataset.map(preprocess_function, batched=True)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=root_dir+"ft_t5/"+setting,
        num_train_epochs=2,
        per_device_train_batch_size=32,
        learning_rate=1e-4,
        weight_decay=0.001,
        logging_dir="./logs",
        logging_steps= 100,
        save_strategy="epoch",
        # save_total_limit=5,
    )

    # Create the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    # Train the model
    trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 8>:12                                                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2432 in from_pretrained   │
│                                                                                                  │
│   2429 │   │   │   │   │   │   "_raise_exceptions_for_missing_entries": False,                   │
│   2430 │   │   │   │   │   │   "_commit_hash": commit_hash,                                      │
│   2431 │   │   │   │   │   }                                                                     │
│ ❱ 2432 │   │   │   │   │   resolved_archive_file = cached_file(pretrained_model_name_or_path, f  │
│   2433 │   │   │   │   │                                                                         │
│   2434 │   │   │   │   │   # Since we set _raise_exceptions_for_missing_entries=False, we don't  │
│   2435 │   │   │   │   │   # result when internet is up, the repo and revision exist, but the f  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:417 in cached_file             │
│                                                                                                  │
│    414 │   user_agent = http_user_agent(user_agent)                                              │
│    415 │   try:                                                                                  │
│    416 │   │   # Load from URL or cache if already cached                                        │
│ ❱  417 │   │   resolved_file = hf_hub_download(                                                  │
│    418 │   │   │   path_or_repo_id,                                                              │
│    419 │   │   │   filename,                                                                     │
│    420 │   │   │   subfolder=None if len(subfolder) == 0 else subfolder,                         │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_validators.py:120 in _inner_fn    │
│                                                                                                  │
│   117 │   │   if check_use_auth_token:                                                           │
│   118 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   119 │   │                                                                                      │
│ ❱ 120 │   │   return fn(*args, **kwargs)                                                         │
│   121 │                                                                                          │
│   122 │   return _inner_fn  # type: ignore                                                       │
│   123                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1364 in hf_hub_download │
│                                                                                                  │
│   1361 │   │   with temp_file_manager() as temp_file:                                            │
│   1362 │   │   │   logger.info("downloading %s to %s", url, temp_file.name)                      │
│   1363 │   │   │                                                                                 │
│ ❱ 1364 │   │   │   http_get(                                                                     │
│   1365 │   │   │   │   url_to_download,                    

In [8]:
# compute perplexity

def shift_tokens_right(input_ids, pad_token_id):
    """
    Shift input ids one token to the right.
    """
    prev_output_tokens = input_ids.clone()
    index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1) - 1).unsqueeze(-1)
    prev_output_tokens[:, 0] = input_ids.gather(1, index_of_eos).squeeze()
    prev_output_tokens[:, 1:] = input_ids[:, :-1]
    return prev_output_tokens


nltk.download('punkt')
#finetuned_model_path = root_dir + "T5_checkpoints/mix_construction/checkpoint-27"
#finetuned_model_path = root_dir + "T5_checkpoints/mix_construction/checkpoint-54"
#finetuned_model_path = root_dir + "T5_checkpoints/wh_island/checkpoint-24"
#finetuned_model_path = root_dir + "T5_checkpoints/wh_island/checkpoint-48"
finetuned_model_path = root_dir + "T5_checkpoints/adjunct_island/checkpoint-32"
#finetuned_model_path = root_dir + "T5_checkpoints/adjunct_island/checkpoint-64"
#finetuned_model_path = root_dir + "T5_checkpoints/complexNP_island/checkpoint-16"
#finetuned_model_path = root_dir + "T5_checkpoints/complexNP_island/checkpoint-32"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = T5ForConditionalGeneration.from_pretrained(finetuned_model_path)
#model = T5ForConditionalGeneration.from_pretrained('T5-base')
model.eval()
model.to(device)
corpus = per_df['full_paragraph'].tolist()
tokenized_inputs = [tokenizer(para) for para in corpus]
input_lengths = [len(x["input_ids"]) for x in tokenized_inputs]
max_source_length = int(np.percentile(input_lengths, 99))
print(max_source_length)
para_perplexity = []

for paragraph in corpus:
    # Tokenize the paragraph into sentences
    sentences = nltk.tokenize.sent_tokenize(paragraph)

    perps = []
    for sentence in sentences:
        # Tokenize the sentence with appropriate padding and truncation
        inputs = tokenizer(sentence, return_tensors="pt").to(device)
        # Create decoder_input_ids
        decoder_input_ids = shift_tokens_right(inputs["input_ids"], tokenizer.pad_token_id)
        inputs["decoder_input_ids"] = decoder_input_ids
        inputs = inputs.to(device)
        input_ids = inputs['input_ids']
        #print(input_ids)
        attention_mask = inputs['attention_mask']
        decoder_input_ids = decoder_input_ids.to(device)
        #print(decoder_input_ids)

        # Get the model's logits
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, labels=decoder_input_ids)
            # print(outputs.loss)
            # outputs = model(**inputs, labels=inputs["input_ids"])
    
        # Compute the cross entropy loss and append to losses
        sent_perplexity = np.exp(outputs.loss.item())
        perps.append(sent_perplexity)
        #losses.append(outputs.loss.item())

    # Compute the paragraph loss
    paragraph_perplexity = np.mean(perps)

    # Compute the perplexity
    #perplexity = np.exp(paragraph_loss)
    para_perplexity.append(paragraph_perplexity)
    print(paragraph)
    print(f"Perplexity of the paragraph: {paragraph_perplexity}")


avg_per = np.mean(para_perplexity)
sd_per = np.std(para_perplexity)
print("Average perplexity:", avg_per, sd_per/np.sqrt(447))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Token indices sequence length is longer than the specified maximum sequence length for this model (812 > 512). Running this sequence through the model will result in indexing errors


407
 Unfortunately , however , and for reasons to be discussed in the following chapter , no rate relationships can be made completely nondiscriminatory as long as all or some of the rates must be set above marginal costs in order to yield adequate revenues . And this fact may explain some of the disagreements among the experts as to the more rational formulas for the apportionment of total costs among different units of service . One such disagreement , which will receive attention in this next chapter , concerns the question whether rates for different kinds of service , in order to avoid the attribute of discrimination , must be made directly proportional to marginal costs , or whether they should be based instead on differences in marginal costs . Here , the choice is that between the horns of a dilemma .
Perplexity of the paragraph: 2.3964034355096695
 One such wife , Dr. Linden says , became disgusted with her weak husband and flurried through a series of extramarital affairs in 

In [ ]:

def formatData(df):
    prefixes = df.iloc[:,:-1].agg(" ".join, axis=1)
    continuation = df["continuation"]
    combined_df = pd.DataFrame({"prefix": prefixes, "continuation": continuation})

    data = {"prefix": [], "continuation": []}
    for i, row in combined_df.iterrows():

        next_words = row["continuation"].split(" ")
        new_prefix = row["prefix"]

        data['prefix'].append(new_prefix)
        data['continuation'].append(next_words[0])

        for i in range(len(next_words)-1):
            new_prefix = new_prefix + " " + next_words[i]
            data['prefix'].append(new_prefix)
            data['continuation'].append(next_words[i+1])
    return data
formatted_data = formatData(eval_df)
print(formatted_data)

{'prefix': ['I know what my brother said that our aunt devoured the cake', 'I know what my brother said that our aunt devoured the cake at', 'I know what my brother said that our aunt devoured the cake at the', 'My neighbor told me what the fireman believes that the dog caught the cat', 'My neighbor told me what the fireman believes that the dog caught the cat in', 'My neighbor told me what the fireman believes that the dog caught the cat in the', 'I know what our assistant believes that the committee delivered the report', 'I know what our assistant believes that the committee delivered the report by', 'I know what our assistant believes that the committee delivered the report by the', 'You remember what I discovered that the water ruined the new carpet', 'You remember what I discovered that the water ruined the new carpet during', 'You remember what I discovered that the water ruined the new carpet during the', 'She told me what the teacher suspects that the star student stole answer

In [ ]:
def getSurprisals(data, tokenizer, model): 
    new_data = {}
    prefixes = data["prefix"]
    continuations = data['continuation']
    # prefixes = ["the dog went to the", "I can hear the dog"]
    # continuations = ["park", "bark"]
    # prefixes = ["The policeman discovered that the convict smuggled a gun", 
    #             "The policeman discovered what the convict smuggled a gun", 
    #             "The policeman discovered that the convict smuggled", 
    #             "The policeman discovered what the convict smuggled"]
    # continuations = [".", ".", ".", "."]
    prefixes = [prefix +"<extra_id_0> " for prefix in prefixes]
    encoded_inputs = tokenizer(prefixes, padding=True, return_tensors="pt")
    encoded_inputs = encoded_inputs.to(device)
    model = model.to(device)
    input_ids = encoded_inputs['input_ids']
    attention_mask = encoded_inputs['attention_mask']
    target_tokens = tokenizer(continuations, truncation=True, return_tensors="pt", max_length=2)['input_ids'][:, :1]
    target_tokens = target_tokens.to(device)
    with torch.no_grad():
            # Generate the next token
            generated = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens = 1)

            # Calculate logits for the generated token
            decoder_input_ids = generated
            logits = model(input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids).logits

            # Select the logits for the generated token only
            next_word_logits = logits[:, -1]

            # Generate predictions
            next_word_probs = torch.softmax(next_word_logits, dim=-1)
            predictions = [tokenizer.decode(torch.argmax(next_word_probs, dim=-1)[i]) for i in range(next_word_logits.size(0))]
            # Calculate surprisals
            true_probs = torch.gather(next_word_probs, 1, target_tokens)
            surprisals = -torch.log(true_probs + 1e-9).cpu().detach().numpy()[:,0]  # Add a small constant for numerical stability
            print(predictions)
    new_data["surprisals"] = surprisals
    new_data['prefix'] = prefixes
    new_data['continuation'] = continuations
    return new_data

finetuned_model_path = root_dir + "ft_t5/checkpoint-2500"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer.padding_side = "left"
model = T5ForConditionalGeneration.from_pretrained(finetuned_model_path)
model.eval()
new_data = getSurprisals(formatted_data, tokenizer, model)
output = pd.DataFrame.from_dict(new_data)
output.to_csv('adj-islands.csv')
display(output)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


['', 'with', 'cigarettes', '']


,surprisals,prefix,continuation
0,0.003455,The policeman discovered that the convict smug...,.
1,2.166620,The policeman discovered what the convict smug...,.
2,5.071034,The policeman discovered that the convict smug...,.
3,0.153939,The policeman discovered what the convict smug...,.


In [ ]:
files.download('adj-islands.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# decoder_input_ids = torch.full_like(input_ids, tokenizer.eos_token_id).to(device)
        # decoder_input_ids = torch.full_like(input_ids, tokenizer.convert_tokens_to_ids("<extra_id_0>")).to(device)
        # output_ids = model.generate(input_ids, attention_mask=attention_mask)
        # logits = model(input_ids, attention_mask=attention_mask, decoder_input_ids=decoder_input_ids).logits
        # # Calculate surprisals
        # next_word_logits = logits[:, -1]
        # next_word_probs = torch.softmax(next_word_logits, dim=-1)
        # predictions = [tokenizer.decode(torch.argmax(next_word_logits, dim=-1)[i]) for i in range(next_word_logits.size(0))]
        # true_probs = torch.gather(next_word_probs, 1, conts)
        # # probs = torch.softmax(logits[:, -1], dim=-1)
        # # true_probs = torch.gather(probs, 1, conts).squeeze()
        # # topk = torch.topk(probs, 1, dim=1)
        # # print(probs.shape)
        # # print(true_probs.shape)
        # # print(topk.indices.shape)
        # # print(tokenizer.batch_decode(topk.indices))
        # print(predictions)
        # print(tokenizer.batch_decode(output_ids))
        # surprisals = -torch.log(true_probs + 1e-9).cpu().detach().numpy()  # Add a small constant for numerical stability